### Compare units that canceled with DDM-like units for this category

####Output: ccm_category_neurons.mat files

In [ ]:
dataPath = fullfile(projectRoot,'data',projectDate,subject);

cEpochList = {'presacc','presaccNoVis','presaccRamp','visPresacc','presaccNoPostsacc'};
dgEpochList = {'Stim','Sacc','Post'};

for i = 1 : length(cEpochList)
    for j = 1 : length(dgEpochList)
        
        dgEpoch = dgEpochList{j};
        cEpoch = cEpochList{i};
        cancelEpoch = 'presaccCancel';




        % LOAD ALL RELEVANT DATA AND LISTS

        % Load Ding and gold neuron classifications
        dg = load(fullfile(dataPath, ['ccm_ddm', dgEpochList{j}, '_neurons']));
        ddm = dg.neurons;


        % Load Classic neuron classifications
        c = load(fullfile(dataPath, ['ccm_',cEpochList{i},'_neurons']));
        classic = c.neurons;


        % Load Cancel Time population 
        ca = load(fullfile(dataPath, ['ccm_',cancelEpoch,'_neurons']));
        classicCancel = ca.neurons;




        [~, iCla, iDdm] = setxor(classic, ddm);
        classicDdm = intersect(classic, ddm);
        classicNoDdm = classic(iCla, :);
        ddmNoClassic = ddm(iDdm, :);


        [~, iCla, iCan] = setxor(classic, cancel);
        classicCancel = intersect(classic, cancel);
        classicNoCancel = classic(iCla, :);
        cancelNoClassic = ddm(iCan, :);


        [~, iCla, iBoth] = setxor(cancel, classicDdm);
        classicCancelNoDdm = cancel(iCla,:);
        classicDdmNoCancel = classicDdm(iBoth,:);
        classicDdmCancel = intersect(classicDdm, cancel);



        % Save the units as population data, to be called later efficiently
        neurons = classicNoDdm;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'NoDdm',dgEpochList{j},'_neurons']), 'neurons')

        neurons = ddmNoClassic;
        save(fullfile(dataPath, ['ccm_ddm',dgEpochList{j},'No',cEpochList{i},'_neurons']), 'neurons')

        neurons = classicCancel;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'Cancel_neurons']), 'neurons')

        neurons = cancelNoClassic;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'NoCancel_neurons']), 'neurons')

        neurons = classicDdm;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'Ddm',dgEpochList{j},'_neurons']), 'neurons')

        neurons = classicCancelNoDdm;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'CancelNoDdm',dgEpochList{j},'_neurons']), 'neurons')

        neurons = classicDdmNoCancel;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'Ddm',dgEpochList{j},'NoCancel_neurons']), 'neurons')

        neurons = classicDdmCancel;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'Ddm',dgEpochList{j},'Cancel_neurons']), 'neurons')


    end
end



### Population spike density functions for each category of neuron

In [ ]:
%% matlab
%cEpochList = {'presacc','presaccNoVis','presaccRamp','visPresacc','presaccNoPostsacc'};
%dgEpochList = {'Stim','Sacc','Post'};

cEpochList = {'presaccNoVis'};
dgEpochList = {'Stim'};

%cEpochList = {'presaccNoVis'};
%dgEpochList = {'Sacc','Post'};

%cEpochList = {'presaccNoVis','presaccRamp','visPresacc','presaccNoPostsacc'};
%dgEpochList = {'Stim','Sacc','Post'};


opt             = ccm_options;
opt.sessionSet  = [];
opt.howProcess  = 'print';
opt.plotFlag    = false;
opt.dataType    = 'neuron';
opt.collapseTarg 	= true;
opt.doStops 	= true;


% Classic w.r.t DDM and canceling
for j = 1 : length(cEpochList)
    for k = 1 : length(dgEpochList)
        
       categoryList = {[cEpochList{j},'NoDdm',dgEpochList{k}]}
      % categoryList = {[cEpochList{j},'Ddm',dgEpochList{k}], [cEpochList{j},'CancelNoDdm',dgEpochList{k}], [cEpochList{j},'Ddm',dgEpochList{k},'NoCancel'], [cEpochList{j},'Ddm',dgEpochList{k},'Cancel'],[cEpochList{j},'NoDdm',dgEpochList{k}]}
   
        for i = 1 : length(categoryList)
            fprintf('\t%s\n',categoryList{i})
            load(fullfile(dataPath, ['ccm_',categoryList{i},'_neurons']))

            opt.categoryName = categoryList{i};
            opt.sessionArray = neurons.sessionID;
            opt.unitArray   = neurons.unit;
            opt.rfList      = neurons.rf;
            opt.hemisphereList      = neurons.hemisphere;


            Data = ccm_population_neuron(subject,projectRoot,projectDate, opt)

            save(fullfile(dataPath, ['ccm_',categoryList{i},'_neuron_population']), 'Data')
            clear Data
        end
    end
end


cEpochList = {'presacc','presaccNoVis','presaccRamp','visPresacc','presaccNoPostsacc'};
dgEpochList = {'Stim','Sacc','Post'};

% Classic w.r.t. canceling
for j = 1 : length(cEpochList)
    categoryList = {[cEpochList{j},'Cancel'], [cEpochList{j},'NoCancel']}

    for i = 1 : length(categoryList)
            fprintf('\t%s\n',categoryList{i})
        load(fullfile(dataPath, ['ccm_',categoryList{i},'_neurons']))

        opt.categoryName = categoryList{i};
        opt.sessionArray = neurons.sessionID;
        opt.unitArray   = neurons.unit;
        opt.rfList      = neurons.rf;
        opt.hemisphereList      = neurons.hemisphere;


        Data = ccm_population_neuron(subject,projectRoot,projectDate, opt)

        save(fullfile(dataPath, ['ccm_',categoryList{i},'_neuron_population']), 'Data')
        clear Data
    end
end


% DDM
for k = 1 : length(dgEpochList)
    categoryList = {['ddm',dgEpochList{k}]}
    for i = 1 : length(categoryList)
            fprintf('\t%s\n',categoryList{i})
        load(fullfile(dataPath, ['ccm_',categoryList{i},'_neurons']))

        opt.categoryName = categoryList{i};
        opt.sessionArray = neurons.sessionID;
        opt.unitArray   = neurons.unit;
        opt.rfList      = neurons.rf;
        opt.hemisphereList      = neurons.hemisphere;


        Data = ccm_population_neuron(subject,projectRoot,projectDate, opt)

        save(fullfile(dataPath, ['ccm_',categoryList{i},'_neuron_population']), 'Data')
        clear Data
    end
end


In [ ]:
subject = 'broca';
categoryList = {'presaccCancel'};
%categoryList = {'presaccDdm'};
%categoryList = {'presaccCancelNoDdm'};
%categoryList = {'presaccDdmNoCancel'};
categoryList = {'presaccDdmCancel'};
%categoryList = {'presaccNoDdm'}
categoryList = {'presaccDdm'}
categoryList = {'ddmNoPresacc', 'presaccDdm'}
categoryList = {'presaccNoDdm', 'presaccNoCancel'}
categoryList = {'presaccNoCancel'}
opt = ccm_population_neuron_plot;

opt.doStops = true;
opt.easyOnly = true;

for i = 1 : length(categoryList)
opt.categoryName = categoryList{i};

    ccm_population_neuron_plot(subject,projectRoot,projectDate,opt)
end


#Continue to Summary Analyses Notebook